In [2]:
import pandas as pd
from IPython.core import display as ICD
from multiprocessing import Pool
import numpy as np
from datetime import timedelta
import re
import os
from sklearn.metrics import pairwise_distances
from sklearn.preprocessing import MultiLabelBinarizer,LabelEncoder
import plotly
import plotly.graph_objects as go
from sklearn.preprocessing import LabelEncoder,MultiLabelBinarizer
from itertools import groupby, product
import json
le = LabelEncoder()

In [15]:
def read_lcm_output(input_name,folder="lcm_results"):
    """Read and restructure LCM output file,rename columns output a df """
    file = f'{folder}/{input_name}'
    df = pd.read_csv(file,header=None)
    df.columns = ["user_ids","support","itemsets","period","property_values"]
    df["period"] = pd.to_datetime(df["period"])
    df["user_ids"] = df.user_ids.apply(lambda x : np.array([int(z) for z in x.split(" ") if z != ""]))
    return df


def read_lcm_output_total(input_name,folder="../lcm_results"):
    """Read and restructure LCM output file,rename columns output a df """
    file = f'{folder}/{input_name}'
    df = pd.read_csv(file,header=None)
    df.columns = ["user_ids","support","itemsets","period","property_values"]
    df["period"] = pd.to_datetime(df["period"])
    df["user_ids"] = df.user_ids.apply(lambda x : np.array([int(z.replace('"',"")) for z in x[1:-1].split(",") if z != ""]))
    return df


# Data preprocessing for echarts 

In [16]:
def format_links(x):
    res = []
    for i in x[0]:
        for idx in range(len(i)-1):
            res.append([i[idx],i[idx+1],x["index"]])
    return res

def make_links(e,index):
    """Product of groups ids for each two consecutive groups periods"""
    prev = e[0]
    for i in e[1:]:
        yield from product(prev,i,[index])
        prev = i

def extract_demographics(input_file):
    demographics = re.findall("\[([A-Z|a-z|_]+),?([A-Z|a-z|_]+)?\]",input_file)[0]
    return  [i for i in demographics if i !=""]
    
def sankey_preprocessing(input_file,stats_folder='../plots/stats',encoders_folder="../plots/encoders",links_folder="../plots/links",groups_folder='../plots/groups',users_demographics = ["DEPARTEMENT","SEX","AGE"],groups_demographics=["STATION_MGT_TYPE","DEPARTEMENT"],users_consecutive_apparition=3):
#     if "M-10-[2-5000]-[AGE]-lcm.out" in input_file :
#         return
    
    demographics = extract_demographics(input_file)
    users = pd.read_csv("../datasets/Total/users.csv",sep=";")
    df = read_lcm_output_total(input_file).sort_values("period").reset_index(drop=True)
    
    file = f'../plots/links/{input_file}'
    mlb = MultiLabelBinarizer(sparse_output=True)
    _df = mlb.fit_transform(df.user_ids.tolist()).astype(bool)
    _df = pd.DataFrame(_df.toarray(),columns=mlb.classes_)
    
    e = _df.sum()
    _df = _df[e[e>1].index]
    _df  = _df.T.apply(lambda x : np.where(x)[0],axis=1)
    e = _df.to_frame()[0].apply(lambda x: list(list(z) for idx,z in groupby(x,lambda y: df.iloc[y].period)))
    e = e[e.apply(lambda x:len(x))>users_consecutive_apparition]
    
    res = []
    e.to_frame().reset_index().apply(lambda x: [res.append(i) for i in make_links(x[0],x["index"])],axis=1)
    links = pd.DataFrame(res)
    links.columns = ["source","target","user_id"]
    links.groupby(["source","target"])["user_id"].apply(lambda x: ','.join(str(i) for i in x)).to_frame().to_csv(file)

    # Users demographics stats  
    file = f'../plots/stats/users/{input_file}'
    stats = {}
    users_stats = links[["user_id"]].drop_duplicates().merge(users,left_on="user_id",right_on="CUST_ID")[users.columns]
    for i in users_demographics:
        b = users_stats.groupby(i).apply(lambda x: {"name":x[i].unique()[0],"value":x.CUST_ID.shape[0],"users":",".join(str(i) for i in x.CUST_ID)}).values
        stats[i] = b.tolist()
        
    with open(file, 'w') as outfile:
        json.dump(stats, outfile)
  
    
    # Users
    file = f'../plots/users/{input_file}'
    users_stats.to_csv(file)
    
    # filter groups to the ones appearing in the links
    file = f'{groups_folder}/{input_file}'
    df_reduced_filtred = df.loc[np.unique(np.union1d(links.source.unique(),links.target.unique()))].dropna()
    df_reduced_filtred['depth'] = le.fit_transform(df_reduced_filtred.period)/df_reduced_filtred.period.nunique()
    df_reduced_filtred['size'] = df_reduced_filtred.user_ids.apply(lambda x : len(x))
    if len(demographics)==1:
        df_reduced_filtred[demographics[0]]= df_reduced_filtred.property_values
    else:
        df_reduced_filtred[demographics]= df_reduced_filtred.property_values.str.split("_",expand=True)
        
    # Encoding items to their initial ID + adding names
    items = pd.read_csv("../datasets/Total/items.csv",encoding="latin",sep=";")
    encoder = LabelEncoder()
    encoder.classes_ = np.load(f'{encoders_folder}/{input_file}.npy')
    return encoder,df_reduced_filtred
    df_reduced_filtred["itemsets_name"] = df_reduced_filtred["itemsets"].apply(lambda x : encoder.inverse_transform([int(i) for i in x.split()]))
    
    z = df_reduced_filtred["itemsets_name"] .apply(lambda x : [items[items.ARTICLE_ID==i].DESCRIPTION.tolist() for i in x ]).to_frame()
    df_reduced_filtred["itemsets_name"] = z.apply(lambda x : "; ".join([i for i in x[0][0]]),axis=1)
    df_reduced_filtred["itemsets"] = df_reduced_filtred["itemsets"].apply(lambda x :":".join(str(i) for i in encoder.inverse_transform([int(i) for i in x.split()])))
    
   
    df_reduced_filtred.to_csv(file)
    
    # Groups demographics stats 
    file = f'../plots/stats/groups/{input_file}'
    stats = {}
    for i in np.intersect1d(groups_demographics,demographics):
        b = df_reduced_filtred.groupby(i).apply(lambda x : {"name":x[i].unique()[0],"value":x.index.shape[0],"groups":",".join(str(i) for i in x.index)}).values
        stats[i]=str(b.tolist())
        
    with open(file, 'w') as outfile:
        json.dump(stats, outfile)
    
    
    print("Done",input_file)


In [17]:
input_file = "9M-5-[1-2001]-[a]-lcm.out"
for i in ["9M-5-[1-2001]-[a]-lcm.out"]:
    print(i,extract_demographics(i))
    try:
        e = sankey_preprocessing(i,users_consecutive_apparition=2)
    except Exception as e:
        print("Error",i,e)

9M-5-[1-2001]-[a]-lcm.out ['a']


In [38]:
d = b.apply(lambda x : a.inverse_transform([int(i) for i in x["itemsets"].split()]),axis=1)

In [67]:
items = pd.read_csv("../datasets/Total/items.csv",encoding="latin",sep=";").drop_duplicates("ARTICLE_ID")
items = items.set_index("ARTICLE_ID")

In [76]:
def items_mapping(x):
    try:
        return items.loc[x].DESCRIPTION
    except:
        print("Item not found",x)
        return "No Name"

In [81]:
d.apply(lambda x : [items_mapping(i) for i in x])

Item not found 6206300138427
Item not found 6206300138427
Item not found 6206300138427
Item not found 100
Item not found 3461039
Item not found 3461039
Item not found 3461039
Item not found 1
Item not found 6206300218860
Item not found 6206300218860
Item not found 5401046
Item not found 5401046
Item not found 5401046
Item not found 99990000
Item not found 3450137
Item not found 5401046
Item not found 5401046
Item not found 5401046
Item not found 6206300138922
Item not found 6206300138922
Item not found 5401046
Item not found 451
Item not found 451
Item not found 451
Item not found 451
Item not found 451
Item not found 6206300219508
Item not found 12520046
Item not found 5401046
Item not found 451
Item not found 12520039
Item not found 24
Item not found 3461040
Item not found 3217770314245
Item not found 5450138
Item not found 3456774836920
Item not found 3412051
Item not found 3461041
Item not found 7804200131952
Item not found 7804200131952
Item not found 26
Item not found 78225003847

3                          [CR SANDWHICH OCEANIQUE (new)]
5                                        [RED BULL 355ML]
9       [CR COOKIE CHOCOLAT LAIT 103 GR, CR SAND PREMIUM]
11      [CR COOKIE CHOCOLAT LAIT 103 GR, CR SANDWICH C...
13                             [No Name, RECHARGE LAVAGE]
                              ...                        
1581                                    [DOLE PECHE SIRO]
1588                         [CR TARTE CITRON MERINGUÉET]
1589               [Lun Salade Accueil, LU POMMES FRITES]
1591                                            [No Name]
1592                   [HOLLYW.BLANCH.MENTH. POLAIRE 14G]
Length: 455, dtype: object

In [75]:
items

,DESCRIPTION
ARTICLE_ID,
3425909001007,RECHARGE LAVAGE TW - A
3425909001083,REC LAVAGE PJE
3124480186188,SCHWEPPES MOJITO 50CL
3559750000893,NEOCLEAN JANTES
5410041005707,LU PIMS ORANGE 150G
...,...
3011449248370,TOTAL LINGET.E.CUIRX16+8
3700232616969,CHARGEUR UNIVER
3609200003339,NA FRUIT CHOC F
